In [21]:
%pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip -q install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.nn.utils.rnn import pack_sequence
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
import pandas as pd
from torch.utils.data import DataLoader, Dataset

In [23]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [24]:
batch_size = 1
industry_size = 32
product_name_size = 20
product_category_size = 26

In [27]:
class LSTMComponent(nn.Module):
    def __init__(self, hidden_dim, output_dim):
        super(LSTMComponent, self).__init__()
        self.lstm = nn.LSTM(input_size=hidden_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, input):
        packed_input = pack_sequence(input, enforce_sorted=False)
        packed_output, (hidden, cell) = self.lstm(packed_input)
        return self.fc(hidden.squeeze(0))

class RecommenderModel(nn.Module):
    def __init__(self, industry_size, max_product_name_size, product_category_size, hidden_dim):
        super(RecommenderModel, self).__init__()

        self.industry_embedding = nn.Embedding(num_embeddings=industry_size, embedding_dim=32)
        self.product_name_embedding = nn.Embedding(num_embeddings=max_product_name_size, embedding_dim=hidden_dim)
        self.product_category_embedding = nn.Embedding(num_embeddings=product_category_size, embedding_dim=hidden_dim)
        self.product_is_implemented_embedding = nn.Embedding(num_embeddings=2, embedding_dim=hidden_dim)

        self.lstm = LSTMComponent(hidden_dim=hidden_dim * 3, output_dim=32)

        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, max_product_name_size)

        self.dropout = nn.Dropout(p=0.2)

    def forward(self, industry, product_name, product_category, product_is_implemented, product_name_size):
        industry_embedded = self.industry_embedding(industry)
        product_name_embedded = self.product_name_embedding(product_name)
        product_category_embedded = self.product_category_embedding(product_category)
        product_is_implemented_embedded = self.product_is_implemented_embedding(product_is_implemented)

        product_embedded = torch.cat([product_name_embedded, product_category_embedded, product_is_implemented_embedded], dim=-1)

        product = self.lstm(product_embedded)

        combined = torch.cat([industry_embedded, product], dim=1)

        x = F.relu(self.fc1(combined))
        x = self.dropout(x)
        output = self.fc2(x)

        output = output[:, :product_name_size]

        return output


In [28]:
model = RecommenderModel(
    industry_size,
    product_name_size,
    product_category_size,
    hidden_dim=256
).to(device)

In [29]:
batch_size = 2
industry_1 = torch.randint(0, industry_size, (batch_size,), device=device)
product_1 = torch.randint(0, product_name_size, (batch_size, 2), device=device)
product_2 = torch.randint(0, product_category_size, (batch_size, 2), device=device)
product_3 = torch.randint(0, 2, (batch_size, 2), device=device)

In [30]:
result = model(industry_1, product_1, product_2, product_3, product_name_size)
result.shape

torch.Size([2, 20])

In [48]:
data = pd.read_csv('dataset.csv')

industry = data["Industry"].tolist()
product_name = data["ProductName"].tolist()
product_category = data["ProductCategory"].tolist()
product_is_implemented = data["ProductIsImplemented"].tolist()

class RecommenderDataset(Dataset):
    def __init__(self, input, output)

SyntaxError: expected ':' (306478829.py, line 9)